Individual NBA player points regression \
Plans
- gather and clean data from many sources
    - save to a csv

In [2]:
from nba_api.stats.endpoints import PlayerGameLogs
from nba_api.stats.static import players
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

In [19]:
df = pd.read_csv("./data/2023_nba_player_stats.csv")

In [ ]:
# Get the player game logs with the correct parameter
player_game_logs = PlayerGameLogs(season_nullable='2023-24')
player_game_logs_df = player_game_logs.get_data_frames()[0]

# createa csv file with the player game logs
player_game_logs_df.to_csv("./data/logs_2023_nba_players.csv", index=False)

In [44]:
print(player_game_logs_df.columns)

Index(['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
       'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2',
       'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK',
       'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK',
       'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK',
       'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK',
       'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK',
       'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
       'WNBA_FANTASY_PTS_RANK', 'AVAILABLE_FLAG', 'MIN_SEC'],
      dtype='object')


In [ ]:
import get_nba_data as nba
# formerly the code in get_nba_data.py was in this box

curry = nba.get_player_data("Stephen Curry")
print(curry)

Player ID: 201939
    game_date  points    minutes  is_home  points_ma  minutes_played_ma  \
0  2024-04-12      33  32.393333     True       33.0          32.393333   
1  2024-04-11      22  36.366667    False       27.5          34.380000   
2  2024-04-09      23  32.316667    False       26.0          33.692222   
3  2024-04-05      28  35.425000    False       26.5          34.125417   
4  2024-04-04      29  30.663333    False       27.0          33.433000   
..        ...     ...        ...      ...        ...                ...   
69 2023-11-01      21  32.330000     True       27.2          32.326333   
70 2023-10-30      42  30.033333    False       31.0          31.439667   
71 2023-10-29      24  31.428333    False       29.0          31.199667   
72 2023-10-27      41  34.833333    False       31.6          32.002000   
73 2023-10-24      27  30.748333     True       31.0          31.874667   

    days_rest  back_to_back  season_avg_points   last_5_games_points  
0         